In [1]:
from cvxopt import matrix, solvers
import scipy.io as sc
import numpy as np
import pandas as pd

In [2]:
def Normalize_std(X_train, X_test):
    mean = np.mean(X_train)
    std_deviation = np.std(X_train)
    X_train = ((X_train - mean)/std_deviation)
    X_test = ((X_test - mean)/std_deviation)
    return X_train, X_test

def Normaize_min_max(X_train, X_test):
    min1 = np.min(X_train)
    max1 = np.max(X_train)
    X_train = ((X_train - min1)/(max1-min1))
    X_test = ((X_test - min1)/(max1-min1))
    return X_train, X_test

In [3]:
def load():
    df = sc.loadmat('spamdata.mat')
    
    X_train = df['xtrain']
    X_test = df['xtest']
    y_train = df['ytrain']
    y_test = df['ytest']

    X_train = np.insert(X_train, 0, 1, axis=1)
    X_test = np.insert(X_test, 0, 1, axis=1)
#     X_train, X_test = Normaize_min_max(X_train, X_test)
     
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load()

In [4]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((500, 58), (500, 1), (2000, 58), (2000, 1))

In [5]:
np.random.seed(13)
w = np.random.normal(0,1,X_train.shape[1]).reshape(-1,1)
epochs=1000

In [6]:
X_train.shape

(500, 58)

In [7]:
def pegesos(X_train, y_train, w, epochs):
    for i in range(epochs):
        np.random.shuffle(X_train)
        X_ = y_train * (X_train @ w)
        
        for j in range(X_train.shape[0]):
            
            if (X_[j][0] < 1) : 
                w = (1 - 1/(1+i))*w + 1/(1+i) * y_train[j] * X_train[j]
            elif (X_[j][0] >= 1):
                w = (1 - 1/(1+i))*w
                
    return w
        
weights = pegesos(X_train, y_train, w, epochs)

In [8]:
error1 = np.abs(((X_train @ w) > 0) * 1 - (y_train>0) * 1).mean()
error2 = np.abs(((X_test @ w) > 0) * 1 - (y_test>0) * 1).mean()

accuracy_train = 1 - error1
accuracy_test = 1 - error2

print("Training Accuracy : ", accuracy_train, "   Test Accuracy : ", accuracy_test)

Training Accuracy :  0.406    Test Accuracy :  0.41400000000000003


In [9]:
from sklearn.metrics import accuracy_score
preds=[]

result= X_test @ w
for i in range(len(result)):
    if(result[i]>1):
        preds.append(1)
    else:
        preds.append(-1)

print(accuracy_score(y_test,preds))

0.4155
